In [1]:
# from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import NearMiss
# from imblearn.metrics import classification_report_imbalanced

In [2]:
# #We first remove the existing CPU-only lightGBM library and clone the latest github repo.
# !rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
# !git clone --recursive https://github.com/Microsoft/LightGBM

In [3]:
# #Next, the Boost development library must be installed.
# !apt-get install -y -qq libboost-all-dev

In [4]:
# %%bash
# cd LightGBM
# rm -r build
# mkdir build
# cd build
# cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
# make -j$(nproc)

In [5]:
# !cd LightGBM/python-package/;python3 setup.py install --precompile

In [6]:
# #Last, carry out some post processing tricks for OpenCL to work properly, and clean up.
# !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
# !rm -r LightGBM

In [7]:
import lightgbm as lgb

In [8]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, confusion_matrix

from sklearn.calibration import CalibratedClassifierCV


from sklearn.metrics import f1_score,precision_recall_curve,roc_curve, recall_score,precision_score

from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')

In [9]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

In [10]:
%%time
Path ='../input/wns-analytic-wizard-preproc/'


train_df = pd.read_csv(Path+'train_preproc.csv')
test_df = pd.read_csv(Path+'test_preproc.csv')

Path ='../input/wns-analyticswizard/'
viewlog_df = pd.read_csv(Path+'train_na17sgz/view_log.csv')
itemdata_df = pd.read_csv(Path+'train_na17sgz/item_data.csv')
subm_df = pd.read_csv(Path+'sample_submission_ipsblct/sample_submission.csv')

targetcol = 'is_click'
target = train_df[targetcol]

CPU times: user 1.9 s, sys: 288 ms, total: 2.18 s
Wall time: 2.9 s


Model

In [11]:
# #Analyse user sessions - start date, end date, number of days 
# temp_date= train_df.groupby(['user_id'])['impression_time'].agg(['min','max'])
# temp_day = train_df.groupby(['user_id'])['impression_day'].agg(['min','max'])

# print(temp_date.head(15))
# print(temp_day.head(15))

In [12]:
# temp_user= train_df.groupby(['user_id'])['impression_time'].agg(['count'])
# print(temp_user['count'].describe())
# print(temp_user['count'].quantile(0.85))
# print(temp_user['count'].quantile(0.95))
# print(temp_user['count'].quantile(0.99))

In [13]:
# train_m_test = set(train_df['user_id'].unique()).difference(set(test_df['user_id'].unique()))
# print(len(train_m_test))
# test_m_train = set(test_df['user_id'].unique()).difference(set(train_df['user_id'].unique()))
# print(len(test_m_train))

# # 74723 - 55078  = 19645 users of train exist in test
# # common users bw train and test : 19645 users

In [14]:
# print(train_df['user_id'].nunique())
# print(test_df['user_id'].nunique())
# viewlog_df['user_id'].nunique()

In [15]:
# viewlog_user= viewlog_df.groupby(['user_id'])['server_time'].agg(['count'])
# print(viewlog_user['count'].describe())
# print(viewlog_user['count'].quantile(0.85))
# print(viewlog_user['count'].quantile(0.95))
# print(viewlog_user['count'].quantile(0.99))

In [16]:
#convert cols to 'categorical' for model
for df in [train_df,test_df]:
    df['app_code'] = df['app_code'].astype('category')

In [17]:
from sklearn.preprocessing  import LabelEncoder

le =LabelEncoder()
le.fit(train_df['os_version'])
train_df['os_version'] = le.transform(train_df['os_version']) 
test_df['os_version'] = le.transform(test_df['os_version']) 
print(train_df['os_version'].unique())
print(test_df['os_version'].unique())

[2 1 0]
[1 2 0]


In [18]:
from tqdm import tqdm
import time

In [19]:


def createlagcol(val_shift,tolag_cols,join_cols,
                       data_shift,
                       data,
                    newcolname_template,
                dateblocknumcol,
                last_block):
    data_shift_proc = data_shift.copy()
    data_shift_proc[dateblocknumcol] = data_shift[dateblocknumcol] + val_shift
    data_shift_proc = data_shift_proc[data_shift_proc[dateblocknumcol]<=last_block]
    coldict = {}
    for col in tolag_cols:
        coldict[col] = getlagcolname(col,newcolname_template,val_shift)
    
    data_shift_proc.rename(columns=coldict,inplace=True)
    
    start = time.time()
    data= pd.merge(data,data_shift_proc, how='left',on=join_cols)
    stop = time.time()
    
    print("Join complete: Execution Time={}".format(stop - start ))
    
    del data_shift_proc
    gc.collect()
    return data


def createlagcols(combined_data,shift_range,indexcols,last_block,dateblocknumcol,target_col):
    aggregator ='mean'

#     train_shift_join=all_data[index_plus_cat_cols].copy()
    for val_shift in tqdm(shift_range):
        for type_id in indexcols:
            print('type_id:',type_id)
            aggcolumnnames = [target_col]
            agggroupbycols = [type_id,dateblocknumcol]

            allcolumns = aggcolumnnames.copy()
            allcolumns.extend(agggroupbycols)
#             train_shift =combined_data.groupby(agggroupbycols).aggregate(aggregator).reset_index()[allcolumns]
            train_shift =combined_data[agggroupbycols+aggcolumnnames].groupby(agggroupbycols).aggregate(aggregator).reset_index()[allcolumns]
            newcolname_template = type_id
            combined_data = createlagcol(val_shift,aggcolumnnames,agggroupbycols,
                           train_shift, combined_data,newcolname_template,
                                         dateblocknumcol,last_block)
            
            
            del train_shift;gc.collect()
            
            # perform smoothing average only for specific type_id and the least shift range
#             if (type_id in exp_smoothing_cols) and (val_shift==min(shift_range):
#                     print('Smoothing start..')
#                     for i,col in enumerate(aggcolumnnames):
#                         lagcol = getlagcolname(col,newcolname_template,val_shift)
#                         combined_data= gen_expsmoothingavg(combined_data[lagcol].fillna(0),agggroupbycols,lagcol,
#                                                            newcolname_template,val_shift)
        
    return combined_data
   

def getlagcolname(col,newcolname_template,val_shift):
    return col+'_'+newcolname_template+'_lag_'+str(val_shift)

In [20]:
#gen date block num
def gendateblocknum(data):
    orig_date_format = '%Y-%m-%d %H:%M:%S'
    data['impression_time']= pd.to_datetime(data['impression_time'],format=orig_date_format)
    data['impression_date'] = data['impression_time'].dt.date
    data['impression_dayofweek'] = data['impression_time'].dt.dayofweek
    data.sort_values('impression_date',inplace=True)
    data['dateblocknum'] = (data['impression_date'] != data['impression_date'].shift()).cumsum()
    data.sort_index(inplace=True)
    
    return data
    


In [21]:
# train_df.sort_values('impression_date',ascending=True).index

In [22]:
# # df_temp = pd.DataFrame()
# # train_df['dateblocknum']= train_df.sort_values('impression_date').cumsum()
# # unique_count = train_df[['impression_date','impression_id']].drop_duplicates().groupby('impression_date').cumcount() + 1
# # unique_count = train_df[['impression_date']].drop_duplicates().cumsum() + 1
# # unique_count.reindex(df.index).ffill()

# # temp_train_df = train_df.sort_values('impression_date')
# # temp_df = (temp_train_df['impression_date'] != temp_train_df['impression_date'].shift()).cumsum()
# print(train_df['dateblocknum'][10000:10012])


In [23]:
# print(train_df['impression_date'].unique())
# print(train_df[['impression_id','impression_date']][10000:10012])
# # print(train_df.sort_values('impression_date',ascending=True)[['impression_id','impression_date']][10000:10012])
# # print(unique_count.shape)
# # print(unique_count.head(100))

# # print(train_df['dateblocknum','impression_date'].head(25))

In [24]:
def getenccolname(colname,cols_agg):
    if 'var' in cols_agg:
        colname ="targetvarenc_"+colname
    elif 'std' in cols_agg:
        colname ="targetstdenc_"+colname 
    elif 'sum' in cols_agg:   
        colname ="targetsumenc_"+colname
    elif 'min' in cols_agg:   
        colname ="targetminenc_"+colname
    elif 'max' in cols_agg:   
        colname ="targetmaxenc_"+colname
    elif 'median' in cols_agg:   
        colname ="targetmedianenc_"+colname
    elif 'count' in cols_agg:   
        colname ="targetcountenc_"+colname
    elif 'iqmean' in cols_agg:   
        colname ="targetiqmeanenc_"+colname
    else:
        colname ="targetenc_"+colname
        
    return colname

def droptargetenccols(train, val,test):
     #remove target encoding fields if present
    targetenccols = [col for col in train.columns if ('targetenc' in col) or ('targetstdenc' in col)]
    train.drop(targetenccols,axis=1,inplace=True)
    val.drop(targetenccols,axis=1,inplace=True)
    targetenccols_test = [col for col in test.columns if ('targetenc' in col) or ('targetstdenc' in col)]
    test.drop(targetenccols_test,axis=1,inplace=True)
    
    return train, val,test

In [25]:
def performsmoothing(averages,targetcolname,train,agg,countSeries,
                     smoothing,min_samples_leaf,noise_level,global_agg_val=None):
    
#         smoothing_v = 1 / (1 + np.exp(-((averages["count"] ) - min_samples_leaf) / smoothing))
    smoothing_v = 1 / (1 + np.exp(-((countSeries - min_samples_leaf) / smoothing)) )

#         print('averages[count] describe:',averages["count"].describe())
    print('smoothing_v describe:',smoothing_v.describe())
    if agg=='mean':
#             global_agg_val = train[targetcolname].mean()
#             print('train[targetcolname] describe:',train[targetcolname].describe())
        if global_agg_val is None:
            global_agg_val = np.nanmean(train[targetcolname].values)
    elif agg=='std':
        global_agg_val= train[targetcolname].std()

    newcol ='newcol'
    if agg=='std':
        print('std before smoothing:',averages[agg].head(25))

    print('averages[agg].shape:',averages[agg].shape)

    averages[newcol] = global_agg_val * (1 - smoothing_v) + averages[agg] * smoothing_v
    
#     averages[newcol] = (countSeries * averages[agg] + smoothing * global_agg_val) / (countSeries + smoothing)
#         smooth = (counts * means + m * mean) / (counts + m)

#     print('averages[newcol] describe before noise:',averages[newcol].describe())

    np.random.seed(42)
    noise = np.random.randn(len(averages[newcol])) * noise_level
    print('len(averages[newcol]:',len(averages[newcol]))
    print('noise:',pd.Series(noise).describe())
    averages[newcol] = averages[newcol] + noise
    print('averages[newcol] describe after noise:',averages[newcol].describe())

    if agg=='std':
        print('std after smoothing:',averages[newcol].head(25))

    del smoothing_v,noise;gc.collect()

    return averages[newcol]

 
def targetencode(train,val,test,catcolnames,targetcolname,
                         smoothing,min_samples_leaf,noise_level):
    #Target Encoding
    
    for i,curcol in enumerate(catcolnames):
        print()
        print('curcol: ',curcol)
        print()
        
        enccol_mean = getenccolname(curcol,'mean')
        enccol_count = getenccolname(curcol,'count')
        
#         print('enccol_mean:',enccol_mean)
#         print('enccol_count:',enccol_count)
        
#         train['group_count'] = train.groupby([curcol]).transform('count')
        averages = train.groupby(curcol).agg({targetcolname: "mean",
                                              curcol: "count"})

        averages.columns =['mean',enccol_count]
        print()
        print('averages mean quantile 0.95:',averages['mean'].quantile(0.05))
        print('averages mean describe:',averages['mean'].describe())
        print('averages mean quantile 0.95:',averages['mean'].quantile(0.95))
        start = time.time()
    
        averages[enccol_mean] = performsmoothing(averages,targetcolname,train,'mean',averages[enccol_count],
                                          smoothing[i],min_samples_leaf[i],noise_level[i],
                                          global_agg_val=None)
        averages.drop(['mean'],axis=1,inplace=True)
        
        train[enccol_mean] = train[curcol].map(averages[enccol_mean])
        val[enccol_mean] = val[curcol].map(averages[enccol_mean])
        test[enccol_mean] = test[curcol].map(averages[enccol_mean])
        
        end = time.time()
        print('update exec time:',end- start)
        
        del averages;gc.collect()
#         print(train[train[enccol_mean].isnull()].shape)
        print('val null:',val[val[enccol_mean].isnull()].shape)
        print('test null:',test[test[enccol_mean].isnull()].shape)
        
#         global_mean= train[enccol_mean].mean()
#         train[enccol_mean].fillna(global_mean,inplace=True)
#         val[enccol_mean].fillna(global_mean,inplace=True)
#         test[enccol_mean].fillna(global_mean,inplace=True)

         
    return train,val,test
   

In [26]:
targetenc_col = 'targetenc_user_id_item_id_viewlog'

In [27]:
# #generate target encoding of item_id column in viewlog
# min_samples_leaf = [1000]
# smoothing = [100]
# noise_level = [0.1]
# # noise_level = [0.0005]


# random_state =888
# n_splits = 5
# folds = StratifiedKFold(n_splits=n_splits, shuffle=False, random_state=random_state)

# for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    
#     print()
#     print('FOLD : ',fold_)
#     print()

#     train_cur = train_df.iloc[trn_idx]
#     val_cur = train_df.iloc[val_idx]
#     target_cur = target.iloc[trn_idx]
#     target_val_cur = target.iloc[val_idx]
    
#     tr_index = train_cur.index
#     val_index = val_cur.index
#     #drop any existing target enc cols
#     train_cur,val_cur,test_df = droptargetenccols(train_cur,val_cur,test_df)    

#     #generate target mean on user id
#     train_user_id_mean = train_cur.groupby(['user_id'])[targetcol].mean().reset_index()
#     print(train_user_id_mean.head())

#     train_user_id_mean.columns = ['user_id','user_id_target_mean']

#     viewlog_cur = pd.merge(viewlog_df, train_user_id_mean, how='left', on='user_id')


#     train_mask = ~(viewlog_cur['user_id_target_mean'].isnull())

#     #generate target encoding on item id in view log
#     viewlog_cur_train = viewlog_cur[train_mask]
#     viewlog_cur_val = viewlog_cur[~train_mask & (viewlog_cur['user_id'].isin(val_cur['user_id'].values))]
#     viewlog_cur_test = viewlog_cur[~train_mask & (viewlog_cur['user_id'].isin(test_df['user_id'].values))]

#     viewlog_cur_train,viewlog_cur_val,viewlog_cur_test = \
#                 targetencode(viewlog_cur_train,viewlog_cur_val,viewlog_cur_test,['item_id'],'user_id_target_mean',
#                              smoothing,min_samples_leaf,noise_level)


#     #generate mean of item id target encoding grouped by user id
#     viewlog_cur_train_mean =viewlog_cur_train.groupby(['user_id'])['targetenc_item_id'].agg(['mean']).reset_index()                                    
#     viewlog_cur_train_mean.columns = ['user_id',targetenc_col]
#     viewlog_cur_val_mean =viewlog_cur_val.groupby(['user_id'])['targetenc_item_id'].agg(['mean']).reset_index()                                    
#     viewlog_cur_val_mean.columns = ['user_id',targetenc_col]
#     viewlog_cur_test_mean =viewlog_cur_test.groupby(['user_id'])['targetenc_item_id'].agg(['mean']).reset_index()                                    
#     viewlog_cur_test_mean.columns = ['user_id',targetenc_col]

#     #merge viewlog_cur_train with train, val and test
#     cols = ['user_id',targetenc_col]
#     train_cur = pd.merge(train_cur, viewlog_cur_train_mean, how='left', on='user_id')
#     val_cur = pd.merge(val_cur, viewlog_cur_val_mean, how='left', on='user_id')
#     test_df = pd.merge(test_df, viewlog_cur_test_mean, how='left', on='user_id')
    
#     train_cur.index = tr_index
#     print(train_cur.index)
#     val_cur.index = val_index

#     enc_cols = [col for col in train_cur.columns if 'targetenc' in col]
#     print('enc cols:',enc_cols)
#     print('save encoding feats...')

#     print('save encoding feats in csv...')
#     train_cur[enc_cols].to_csv('train_targetenc_feats'+str(fold_)+'.csv')
#     val_cur[enc_cols].to_csv('val_targetenc_feats'+str(fold_)+'.csv')
#     test_df[enc_cols].to_csv('test_targetenc_feats'+str(fold_)+'.csv')

# #drop any existing target enc cols
# train_cur,val_cur,test_df = droptargetenccols(train_cur,val_cur,test_df)    


In [28]:
def getenc():
    tr_encs = []
    val_encs = []
    test_encs = []
    
    Path=''

    for i in range(0,5):
        cur_tr_enc = pd.read_csv(Path+'train_targetenc_feats'+str(i)+'.csv',index_col=0)
        cur_val_enc = pd.read_csv(Path+'val_targetenc_feats'+str(i)+'.csv',index_col=0)

        tr_encs += [cur_tr_enc]
        val_encs +=[ cur_val_enc]

        test_encs += [pd.read_csv(Path+'test_targetenc_feats'+str(i)+'.csv',index_col=0)]
        print('read complete for:',i)
        
    return tr_encs,val_encs,test_encs

In [29]:
# tr_encs, val_encs,test_encs = getenc()

In [30]:

train_df['istrain']=1
test_df['istrain']=0

combined = pd.concat([train_df,test_df],axis=0)
combined.sort_values('impression_time',inplace=True)

In [31]:
combined = gendateblocknum(combined)
last_block = combined['dateblocknum'].max()

In [32]:
#create lag columns
def genlagcols(lags,index_cols,tolagcol,data):
    for i in lags:
        print('lag:',i)
        data['_'.join(['lag', str(i)])] = data.groupby(index_cols)[tolagcol].transform(lambda x:x.shift(i).mean()) 
    return data
#     data['_'.join(['item-week_shifted-', str(i)])].fillna(data['_'.join(['item-week_shifted-', str(i)])].mode()[0], inplace=True)


In [33]:
# shift_range = list(range(5,7))
shift_range = [5,6,7,14,15,21,22,28]
indexcols = ['app_code']
dateblocknumcol = 'dateblocknum'
# combined = createlagcols(combined,shift_range,indexcols,last_block,dateblocknumcol,targetcol)
# combined = genlagcols(shift_range,indexcols,targetcol,combined)

marker = train_df.shape[0]
train_df = combined[combined['istrain']==1]
test_df = combined[combined['istrain']==0]

del train_df['istrain'],test_df['istrain'],combined,test_df[targetcol]

In [34]:
lagcols = [col for col in train_df.columns if 'lag_' in col]
dispcols = ['impression_id','impression_time','dateblocknum']+lagcols+indexcols


In [35]:
# train_df.drop(lagcols,inplace=True,axis=1)
# test_df.drop(lagcols,inplace=True,axis=1)

In [36]:

# train_df[train_df['dateblocknum']>28][dispcols].head(25)

# # train_df[(train_df['app_code'].isin([522])) & (train_df['dateblocknum'].isin([22,23]))][dispcols]
# # train_df[(train_df['app_code'].isin([260,44,242])) & (train_df['dateblocknum'].isin([23,24]))][lagcols+indexcols+[targetcol]]

In [37]:
# # test_df[test_df['app_code']==242][dispcols].head(25)
# test_df[dispcols].head(25)

In [38]:
# #analysis of impression date on train dataset
# # temp = train_df.groupby('user_id',
# #                         'impression_year','impression_month','impression_day'
# #                         'impression_day','impression_hour')['impression_day'].count()
# print(list(train_df.columns))
# # temp = train_df.groupby(['user_id'])['dateblocknum'].count()
# # temp = train_df.groupby(['user_id','dateblocknum'])['impression_hour'].count()
# temp = train_df.groupby(['user_id','dateblocknum','impression_hour'])['impression_id'].count()
# # print(temp.head(25))
# print(train_df['dateblocknum'].unique())
# print(train_df['user_id'].nunique())
# print(temp.count())
# print(temp[temp>1].count())

In [39]:
# orig_date_format = '%Y-%m-%d %H:%M:%S'
# viewlog_df['server_time']= pd.to_datetime(viewlog_df['server_time'],format=orig_date_format)
# viewlog_df['server_date'] = viewlog_df['server_time'].dt.date
# viewlog_df['server_day'] = viewlog_df['server_time'].dt.day
# viewlog_df['server_month'] = viewlog_df['server_time'].dt.month
# viewlog_df['server_hour'] = viewlog_df['server_time'].dt.hour

In [40]:
# print(viewlog_df['server_date'].min())
# print(viewlog_df['server_date'].max())

# print('view log total:',viewlog_df.shape[0])
# print('session_id unique:',viewlog_df['session_id'].nunique())
# print('item id unique:',viewlog_df['item_id'].nunique())

# temp = viewlog_df.groupby(['user_id','item_id','server_time'])['device_type'].count()
# print('grouping count:',temp.count())
# print('grouping filtered count:',temp[temp>1].count())
# print('grouping filtered head:',temp[temp>1].head(15))
# viewlog_df.head()

In [41]:
# itemdata_df.head()

In [42]:
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(tokenizer=lambda x: x.split(' '))

In [43]:
#merge view log with item description table
viewlog_merged = pd.merge(viewlog_df, itemdata_df, how='left', on='item_id')

In [44]:
temp  = viewlog_merged.sort_values(['user_id','session_id'])
temp.head(100)


,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
2627295,2018-12-04 07:19:00,android,86373,0,116073,4181.0,17.0,39.0,151.0,124.0
1355010,2018-11-11 10:09:00,android,143955,0,89373,2496.0,17.0,8.0,113.0,10025.0
1355056,2018-11-11 10:10:00,android,143955,0,10379,1238.0,16.0,17.0,272.0,60.0
1355084,2018-11-11 10:10:00,android,143955,0,10379,1238.0,16.0,17.0,272.0,60.0
1355117,2018-11-11 10:11:00,android,143955,0,89373,2496.0,17.0,8.0,113.0,10025.0
1355242,2018-11-11 10:13:00,android,143955,0,40039,92160.0,1.0,7.0,308.0,6932.0
1355271,2018-11-11 10:13:00,android,143955,0,40039,92160.0,1.0,7.0,308.0,6932.0
1355329,2018-11-11 10:14:00,android,143955,0,88013,63744.0,1.0,42.0,30.0,9755.0
1355387,2018-11-11 10:15:00,android,143955,0,2715,1601.0,14.0,61.0,159.0,7093.0
1355488,2018-11-11 10:16:00,android,143955,0,2715,1601.0,14.0,61.0,159.0,7093.0


In [45]:
print(viewlog_merged['session_id'].nunique())
print(viewlog_merged['item_id'].nunique())
print(viewlog_merged['session_id'].shape)

1014970
126708
(3118622,)


In [46]:
print(viewlog_merged['item_price'].quantile(0.01))
print(viewlog_merged['item_price'].quantile(0.10))
print(viewlog_merged['item_price'].quantile(0.15))
print(viewlog_merged['item_price'].describe())
print(viewlog_merged['item_price'].quantile(0.85))
print(viewlog_merged['item_price'].quantile(0.9))
print(viewlog_merged['item_price'].quantile(0.95))
print(viewlog_merged['item_price'].quantile(0.99))
print(viewlog_merged['item_price'].quantile(0.995))
print(viewlog_merged['item_price'].quantile(0.999))

179.0
640.0
864.0
count    3.116840e+06
mean     1.060242e+04
std      2.459572e+04
min      5.000000e+00
25%      1.369000e+03
50%      2.873000e+03
75%      8.663000e+03
max      1.340800e+06
Name: item_price, dtype: float64
15680.0
24640.0
47936.0
135680.0
179878.0
231040.0


In [47]:
#Create item price bin
viewlog_merged['item_price_bin'] = pd.cut(viewlog_merged['item_price'], 
                                   [0,100,500,1000,5000,10000,25000,50000,100000,500000,999999999999], 
                                   labels=False, retbins=False, right=False)

print(pd.Series(viewlog_merged['item_price_bin'].describe()))
print(viewlog_merged[viewlog_merged['item_price']<100][['item_price_bin','item_price']].head(15))
viewlog_merged[viewlog_merged['item_price']>500000][['item_price_bin','item_price']].head(15)

count    3.116840e+06
mean     3.500282e+00
std      1.454528e+00
min      0.000000e+00
25%      3.000000e+00
50%      3.000000e+00
75%      4.000000e+00
max      9.000000e+00
Name: item_price_bin, dtype: float64
      item_price_bin  item_price
131              0.0        76.0
244              0.0        50.0
335              0.0        50.0
639              0.0        80.0
1114             0.0        80.0
1126             0.0        80.0
1446             0.0        80.0
1474             0.0        80.0
1495             0.0        50.0
1637             0.0        64.0
2193             0.0        80.0
2232             0.0        80.0
2245             0.0        78.0
2786             0.0        80.0
2964             0.0        80.0


,item_price_bin,item_price
11097,9.0,516800.0
104979,9.0,544000.0
127431,9.0,512000.0
127499,9.0,512000.0
340564,9.0,870400.0
340573,9.0,870400.0
441050,9.0,569600.0
542740,9.0,790899.0
542766,9.0,790899.0
542826,9.0,790899.0


In [48]:
# temp = viewlog_merged.groupby(['user_id'])['device_type']\
#                                                 .agg(['nunique']).reset_index()
# print(temp.describe())
# print(temp[temp['nunique']>1].shape)

In [49]:
# catcols_viewlog = ['category_1','category_2','category_3','item_id','product_type']
catcols_viewlog = ['category_1','category_2','category_3',
                   'item_id','product_type','item_price_bin']

def get_row_str_cat(x):
    x = [str(x) for x in x]
    return ' '.join(x)
viewlog_texts = viewlog_merged[['user_id']+catcols_viewlog].groupby('user_id').agg(get_row_str_cat).reset_index()
print(viewlog_texts.columns)

viewlog_texts.columns = ['user_id','category_1_texts',
                            'category_2_texts','category_3_texts',
                            'item_id_texts','product_type_texts','item_price_bin_texts'
                        ]
# viewlog_texts.columns = ['user_id','item_id_texts']
viewlog_texts.head(10)


Index(['user_id', 'category_1', 'category_2', 'category_3', 'item_id',
       'product_type', 'item_price_bin'],
      dtype='object')


,user_id,category_1_texts,category_2_texts,category_3_texts,item_id_texts,product_type_texts,item_price_bin_texts
0,0,14.0 14.0 14.0 14.0 14.0 14.0 7.0 13.0 4.0 7.0...,29.0 29.0 29.0 29.0 29.0 29.0 24.0 58.0 74.0 2...,199.0 199.0 199.0 199.0 199.0 199.0 126.0 173....,44459 44459 44459 44459 44459 44459 40228 5832...,7363.0 7363.0 7363.0 7363.0 7363.0 7363.0 8351...,3.0 3.0 3.0 3.0 3.0 3.0 1.0 3.0 3.0 1.0 3.0 3....
1,1,11.0 7.0 12.0 8.0 0.0 15.0 11.0 13.0,76.0 24.0 57.0 40.0 6.0 0.0 76.0 78.0,147.0 100.0 44.0 84.0 277.0 214.0 147.0 134.0,64835 27574 22411 91179 126431 67259 40952 6615,7455.0 5168.0 7391.0 4924.0 4552.0 3303.0 7738...,4.0 3.0 3.0 5.0 3.0 1.0 3.0 3.0
2,2,17.0 17.0 1.0 17.0 17.0 1.0 17.0 14.0 1.0 1.0 ...,26.0 26.0 28.0 26.0 8.0 12.0 55.0 62.0 12.0 64...,233.0 233.0 314.0 225.0 279.0 301.0 70.0 324.0...,127236 127236 61874 31772 12208 42173 33074 89...,8127.0 8127.0 10097.0 2874.0 9041.0 7455.0 743...,5.0 5.0 4.0 3.0 3.0 3.0 2.0 3.0 5.0 7.0 3.0 4....
3,3,1.0 1.0 1.0 14.0 14.0 0.0 0.0 14.0,42.0 42.0 42.0 62.0 62.0 6.0 6.0 62.0,157.0 157.0 157.0 112.0 112.0 50.0 50.0 112.0,27934 27934 27934 3942 3942 26074 26074 3942,6659.0 6659.0 6659.0 491.0 491.0 4805.0 4805.0...,5.0 5.0 5.0 3.0 3.0 2.0 2.0 3.0
4,4,9.0 14.0,3.0 61.0,27.0 159.0,76717 38231,234.0 4426.0,7.0 3.0
5,5,1.0,28.0,336.0,90842,9187.0,4.0
6,6,14.0 11.0 14.0 9.0 14.0 9.0 14.0 17.0 10.0 4.0...,29.0 70.0 61.0 9.0 62.0 9.0 62.0 8.0 43.0 74.0...,257.0 322.0 132.0 244.0 315.0 244.0 315.0 159....,6934 75693 16770 121033 62098 32155 62098 4649...,1584.0 7062.0 2840.0 10412.0 1397.0 10412.0 13...,3.0 3.0 3.0 3.0 5.0 3.0 5.0 3.0 3.0 3.0 3.0 3....
7,7,14.0 10.0 13.0 17.0 4.0 17.0,29.0 43.0 5.0 39.0 38.0 8.0,199.0 187.0 22.0 252.0 62.0 279.0,40691 3359 26512 16164 11871 111805,4770.0 6077.0 1671.0 4741.0 10412.0 1961.0,2.0 6.0 5.0 3.0 3.0 3.0
8,8,4.0 17.0 0.0 17.0 12.0 17.0 17.0 17.0 17.0 15....,38.0 69.0 45.0 69.0 57.0 69.0 26.0 26.0 8.0 79...,19.0 194.0 73.0 194.0 54.0 194.0 233.0 233.0 2...,54788 91322 51866 91322 36387 91322 68615 6861...,3893.0 2874.0 6718.0 2874.0 6647.0 2874.0 2077...,7.0 2.0 3.0 2.0 3.0 2.0 3.0 3.0 3.0 3.0 6.0 5....
9,9,1.0 1.0 11.0 16.0 13.0 17.0 16.0 13.0 13.0 13....,42.0 42.0 51.0 4.0 5.0 26.0 4.0 78.0 5.0 78.0 ...,157.0 157.0 329.0 98.0 65.0 233.0 98.0 205.0 6...,88199 88199 104235 2391 30086 101298 2391 1204...,6598.0 6598.0 9865.0 8045.0 3765.0 9749.0 8045...,5.0 5.0 3.0 3.0 5.0 4.0 3.0 4.0 5.0 3.0 3.0 3....


In [50]:
cur_catcols_viewlog = ['category_1','category_2','category_3',
                       'item_price_bin']
for col in cur_catcols_viewlog:
    print(col)
    X_tfidf = tf.fit_transform(viewlog_texts[col+'_texts'])
#     X_tfidf = pd.DataFrame(X_tfidf.todense(), columns = ['tfidf_dense_'+str(x) for x in tf.vocabulary_.values()])
    X_tfidf = pd.DataFrame(X_tfidf.todense(), columns = ['tfidf_dense_'+col+str(x) for x in tf.vocabulary_.values()])
    print(X_tfidf.shape)
    viewlog_texts = pd.concat([viewlog_texts, X_tfidf], axis=1)
    
viewlog_texts.head(10)

category_1
(89157, 18)
category_2
(89157, 80)
category_3
(89157, 336)
item_price_bin
(89157, 11)


,user_id,category_1_texts,category_2_texts,category_3_texts,item_id_texts,product_type_texts,item_price_bin_texts,tfidf_dense_category_16,tfidf_dense_category_114,tfidf_dense_category_15,tfidf_dense_category_111,tfidf_dense_category_19,tfidf_dense_category_18,tfidf_dense_category_11,tfidf_dense_category_113,tfidf_dense_category_17,tfidf_dense_category_13,tfidf_dense_category_14,tfidf_dense_category_115,tfidf_dense_category_10,tfidf_dense_category_116,tfidf_dense_category_12,tfidf_dense_category_117,tfidf_dense_category_112,tfidf_dense_category_110,tfidf_dense_category_222,tfidf_dense_category_217,tfidf_dense_category_253,tfidf_dense_category_271,tfidf_dense_category_277,tfidf_dense_category_29,tfidf_dense_category_266,tfidf_dense_category_236,tfidf_dense_category_257,tfidf_dense_category_232,tfidf_dense_category_243,tfidf_dense_category_260,tfidf_dense_category_276,tfidf_dense_category_226,tfidf_dense_category_229,tfidf_dense_category_273,tfidf_dense_category_252,tfidf_dense_category_234,tfidf_dense_category_255,tfidf_dense_category_20,tfidf_dense_category_275,tfidf_dense_category_219,tfidf_dense_category_221,tfidf_dense_category_24,tfidf_dense_category_250,tfidf_dense_category_258,tfidf_dense_category_274,tfidf_dense_category_223,tfidf_dense_category_259,tfidf_dense_category_233,tfidf_dense_category_230,tfidf_dense_category_214,tfidf_dense_category_216,tfidf_dense_category_225,tfidf_dense_category_22,tfidf_dense_category_246,tfidf_dense_category_262,tfidf_dense_category_241,tfidf_dense_category_235,tfidf_dense_category_25,tfidf_dense_category_278,tfidf_dense_category_237,tfidf_dense_category_264,tfidf_dense_category_267,tfidf_dense_category_238,tfidf_dense_category_213,tfidf_dense_category_231,tfidf_dense_category_21,tfidf_dense_category_239,tfidf_dense_category_240,tfidf_dense_category_210,tfidf_dense_category_244,tfidf_dense_category_265,tfidf_dense_category_251,tfidf_dense_category_247,tfidf_dense_category_211,tfidf_dense_category_26,tfidf_dense_category_228,tfidf_dense_category_263,tfidf_dense_category_245,tfidf_dense_category_268,tfidf_dense_category_227,tfidf_dense_category_249,tfidf_dense_category_27,tfidf_dense_category_212,tfidf_dense_category_269,tfidf_dense_category_270,tfidf_dense_category_279,tfidf_dense_category_224,tfidf_dense_category_23,tfidf_dense_category_242,tfidf_dense_category_254,tfidf_dense_category_220,tfidf_dense_category_218,tfidf_dense_category_261,...,tfidf_dense_category_3103,tfidf_dense_category_3124,tfidf_dense_category_3174,tfidf_dense_category_3295,tfidf_dense_category_333,tfidf_dense_category_3283,tfidf_dense_category_3181,tfidf_dense_category_3134,tfidf_dense_category_366,tfidf_dense_category_3251,tfidf_dense_category_3228,tfidf_dense_category_3293,tfidf_dense_category_3120,tfidf_dense_category_343,tfidf_dense_category_3144,tfidf_dense_category_329,tfidf_dense_category_3165,tfidf_dense_category_30,tfidf_dense_category_3311,tfidf_dense_category_3291,tfidf_dense_category_3236,tfidf_dense_category_3263,tfidf_dense_category_347,tfidf_dense_category_392,tfidf_dense_category_3314,tfidf_dense_category_3242,tfidf_dense_category_3162,tfidf_dense_category_381,tfidf_dense_category_38,tfidf_dense_category_3189,tfidf_dense_category_3231,tfidf_dense_category_385,tfidf_dense_category_3117,tfidf_dense_category_3202,tfidf_dense_category_322,tfidf_dense_category_371,tfidf_dense_category_357,tfidf_dense_category_3129,tfidf_dense_category_3114,tfidf_dense_category_330,tfidf_dense_category_3316,tfidf_dense_category_3204,tfidf_dense_category_382,tfidf_dense_category_3163,tfidf_dense_category_3261,tfidf_dense_category_3100,tfidf_dense_category_3317,tfidf_dense_category_3277,tfidf_dense_category_3268,tfidf_dense_category_365,tfidf_dense_category_3104,tfidf_dense_category_3108,tfidf_dense_category_3301,tfidf_dense_category_359,tfidf_dense_category_3267,tfidf_dense_category_368,tfidf_dense_category_3139,tfidf_dense_category_3238,tfidf_dense_category_320,tfidf_dense_category_327,tfidf_dense_category_3154,tfidf_dense_

In [51]:
# train_df = pd.merge(train_df, viewlog_texts, how='left', on='user_id')
# test_df = pd.merge(test_df, viewlog_texts, how='left', on='user_id')

In [52]:
# viewlog_texts_copy = viewlog_texts.copy()

In [53]:
# viewlog_texts = viewlog_texts_copy.copy()

In [54]:
# viewlog_texts = viewlog_texts_copy.sample(n=1000)
# print(viewlog_texts.shape)
# print(viewlog_texts.head())

In [55]:

# for col in catcols_viewlog:
#     print(col)
#     X_tfidf = tf.fit_transform(train_df[col+'_texts'])
# #     X_tfidf = pd.DataFrame(list(X_tfidf), columns = ['tfidf_dense_item_id'])
# #     X_tfidf = pd.DataFrame(X_tfidf.todense(), columns = ['tfidf_dense_'+str(x) for x in tf.vocabulary_.values()])
# #     train_df = pd.concat([train_df, X_tfidf], axis=1)
#     train_sparse = pd.SparseDataFrame(X_tfidf,
#                              columns=['tfidf_'+col for col in tf.get_feature_names()], 
#                              default_fill_value=0)    
#     print(train_sparse.shape)
#     print(list(train_sparse.columns)[0:25])


In [56]:
def get_top_tf_idf_words(response, feature_names,top_n=10):
    sorted_data = np.sort(response.data)[:-(top_n+1):-1]
    sorted_nzs = np.argsort(response.data)[:-(top_n+1):-1]
    return feature_names[response.indices[sorted_nzs]],sorted_data

def get_top_fulldata(responses,feature_names,top_n=10):
    topdata =[]
    topfeatsdata =[]
    topvaluesdata =[]
    for response in tqdm(responses):
        topfeats_topn_size =  np.full(top_n, np.nan) 
        topvalues_topn_size =  np.full(top_n, np.nan) 
        topfeats,topvalues = get_top_tf_idf_words(response,feature_names,top_n)
#         print('topfeats:',topfeats)
#         print('topvalues:',topvalues)
        topfeats_topn_size[:len(topfeats)] = topfeats[:]
        topvalues_topn_size[:len(topfeats)] = topvalues[:]
        topfeatsdata += [topfeats_topn_size]
        topvaluesdata += [topvalues_topn_size]
        
#         topdata += [(topfeats_topn_size,topvalues_topn_size)]
        
#     topdata = np.array(topdata)
    topfeatsdata = np.array(topfeatsdata)
    topvaluesdata = np.array(topvaluesdata)
    print(topfeatsdata[0:5,:])
    print(topvaluesdata[0:5,:])
    return topfeatsdata,topvaluesdata

In [57]:
cur_catcols_viewlog = ['item_id','product_type']
    
top_n = 10 

for col in cur_catcols_viewlog:
    print(col)
    X_tfidf = tf.fit_transform(viewlog_texts[col+'_texts'])
    feature_names = np.array(tf.get_feature_names())
    topfeatsdata,topvaluesdata = get_top_fulldata(X_tfidf,feature_names,top_n)
    
    #generate dataframe columns
    for i in range(topfeatsdata.shape[1]):
        namecol = 'tfidf_'+ col + '_name_top_' + str(i+1)
        valuecol = 'tfidf_'+ col + '_value_top_' + str(i+1)
        viewlog_texts[namecol] = topfeatsdata[:,i]
        viewlog_texts[valuecol] = topvaluesdata[:,i]
        
print(viewlog_texts.shape)
viewlog_texts.head(10)

item_id


89157it [00:07, 12088.18it/s]


[[ 44459.  66401.  60603.  40228. 116073.  40039. 126865.  58320.   2715.
   10379.]
 [  6615.  67259. 126431.  22411.  91179.  40952.  27574.  64835.     nan
      nan]
 [127236.  74019.  61874.  49603.  42540.  98317.  84331.  78389.  30393.
   42173.]
 [  3942.  27934.  26074.     nan     nan     nan     nan     nan     nan
      nan]
 [ 76717.  38231.     nan     nan     nan     nan     nan     nan     nan
      nan]]
[[0.59311438 0.46530838 0.33586195 0.31706562 0.28674485 0.15033689
  0.1433453  0.11639839 0.11578336 0.11041494]
 [0.55341509 0.40184096 0.36048178 0.34804308 0.33712367 0.28189143
  0.22733863 0.19072622        nan        nan]
 [0.43598025 0.2747378  0.1716967  0.16297067 0.15214237 0.14678801
  0.14369518 0.13899785 0.12059268 0.11354376]
 [0.61553276 0.61553276 0.49217764        nan        nan        nan
         nan        nan        nan        nan]
 [0.72381198 0.68999726        nan        nan        nan        nan
         nan        nan        nan        nan]

89157it [00:07, 11996.41it/s]


[[ 7363. 10457.  2823.   124.  8351.  6932.  9344.  5358.  7093.    60.]
 [10112.  7391.  3303.  4552.  5168.  7738.  4924.  7455.    nan    nan]
 [ 5293.  8127.  4246.  9597.  9870.  9466.  6239. 10097.  7455.  1961.]
 [ 4805.  6659.   491.    nan    nan    nan    nan    nan    nan    nan]
 [  234.  4426.    nan    nan    nan    nan    nan    nan    nan    nan]]
[[0.59974496 0.45721831 0.39350191 0.27201459 0.25443562 0.16877254
  0.14908791 0.13851284 0.12454999 0.12002171]
 [0.51029779 0.43149938 0.42303146 0.35314134 0.28354481 0.26661871
  0.25084776 0.18796871        nan        nan]
 [0.34627782 0.30708526 0.21718674 0.20303636 0.17565142 0.17232633
  0.15394973 0.14460151 0.12563841 0.1224568 ]
 [0.65945362 0.54065181 0.52231843        nan        nan        nan
         nan        nan        nan        nan]
 [0.71852233 0.69550389        nan        nan        nan        nan
         nan        nan        nan        nan]]
(89157, 492)


,user_id,category_1_texts,category_2_texts,category_3_texts,item_id_texts,product_type_texts,item_price_bin_texts,tfidf_dense_category_16,tfidf_dense_category_114,tfidf_dense_category_15,tfidf_dense_category_111,tfidf_dense_category_19,tfidf_dense_category_18,tfidf_dense_category_11,tfidf_dense_category_113,tfidf_dense_category_17,tfidf_dense_category_13,tfidf_dense_category_14,tfidf_dense_category_115,tfidf_dense_category_10,tfidf_dense_category_116,tfidf_dense_category_12,tfidf_dense_category_117,tfidf_dense_category_112,tfidf_dense_category_110,tfidf_dense_category_222,tfidf_dense_category_217,tfidf_dense_category_253,tfidf_dense_category_271,tfidf_dense_category_277,tfidf_dense_category_29,tfidf_dense_category_266,tfidf_dense_category_236,tfidf_dense_category_257,tfidf_dense_category_232,tfidf_dense_category_243,tfidf_dense_category_260,tfidf_dense_category_276,tfidf_dense_category_226,tfidf_dense_category_229,tfidf_dense_category_273,tfidf_dense_category_252,tfidf_dense_category_234,tfidf_dense_category_255,tfidf_dense_category_20,tfidf_dense_category_275,tfidf_dense_category_219,tfidf_dense_category_221,tfidf_dense_category_24,tfidf_dense_category_250,tfidf_dense_category_258,tfidf_dense_category_274,tfidf_dense_category_223,tfidf_dense_category_259,tfidf_dense_category_233,tfidf_dense_category_230,tfidf_dense_category_214,tfidf_dense_category_216,tfidf_dense_category_225,tfidf_dense_category_22,tfidf_dense_category_246,tfidf_dense_category_262,tfidf_dense_category_241,tfidf_dense_category_235,tfidf_dense_category_25,tfidf_dense_category_278,tfidf_dense_category_237,tfidf_dense_category_264,tfidf_dense_category_267,tfidf_dense_category_238,tfidf_dense_category_213,tfidf_dense_category_231,tfidf_dense_category_21,tfidf_dense_category_239,tfidf_dense_category_240,tfidf_dense_category_210,tfidf_dense_category_244,tfidf_dense_category_265,tfidf_dense_category_251,tfidf_dense_category_247,tfidf_dense_category_211,tfidf_dense_category_26,tfidf_dense_category_228,tfidf_dense_category_263,tfidf_dense_category_245,tfidf_dense_category_268,tfidf_dense_category_227,tfidf_dense_category_249,tfidf_dense_category_27,tfidf_dense_category_212,tfidf_dense_category_269,tfidf_dense_category_270,tfidf_dense_category_279,tfidf_dense_category_224,tfidf_dense_category_23,tfidf_dense_category_242,tfidf_dense_category_254,tfidf_dense_category_220,tfidf_dense_category_218,tfidf_dense_category_261,...,tfidf_dense_category_3316,tfidf_dense_category_3204,tfidf_dense_category_382,tfidf_dense_category_3163,tfidf_dense_category_3261,tfidf_dense_category_3100,tfidf_dense_category_3317,tfidf_dense_category_3277,tfidf_dense_category_3268,tfidf_dense_category_365,tfidf_dense_category_3104,tfidf_dense_category_3108,tfidf_dense_category_3301,tfidf_dense_category_359,tfidf_dense_category_3267,tfidf_dense_category_368,tfidf_dense_category_3139,tfidf_dense_category_3238,tfidf_dense_category_320,tfidf_dense_category_327,tfidf_dense_category_3154,tfidf_dense_category_3257,tfidf_dense_category_3110,tfidf_dense_category_3156,tfidf_dense_category_3198,tfidf_dense_category_332,tfidf_dense_category_399,tfidf_dense_category_3329,tfidf_dense_category_372,tfidf_dense_category_3193,tfidf_dense_category_3215,tfidf_dense_category_3248,tfidf_dense_category_3321,tfidf_dense_category_3211,tfidf_dense_category_3300,tfidf_dense_category_3334,tfidf_dense_category_3288,tfidf_dense_category_3172,tfidf_dense_category_3136,tfidf_dense_category_3217,tfidf_dense_category_3205,tfidf_dense_category_383,tfidf_dense_category_380,tfidf_dense_category_370,tfidf_dense_category_3313,tfidf_dense_category_3280,tfidf_dense_category_3161,tfidf_dense_category_3126,tfidf_dense_category_325,tfidf_dense_item_price_bin3,tfidf_dense_item_price_bin1,tfidf_dense_item_price_bin7,tfidf_dense_item_price_bin4,tfidf_dense_item_price_bin5,tfidf_dense_item_price_bin2,tfidf_dense_item_price_bin8,tfidf_dense_item_price_bin6,tfidf_dense_item_price_bin0,tfidf_dense_item_price_bin10,tfidf_dense_item_price_bin9,tfidf_

In [58]:
# cur_features = ['app_code', 'impression_day', 'impression_hour', 'impression_month', 
#  'impression_year', 'is_4G', 'user_id']
# for col in cur_features:
#     train_sparse[col] = train_df[col]

In [59]:
for col in catcols_viewlog:
    viewlog_texts = viewlog_texts.drop(col+'_texts',axis=1)
train_df = pd.merge(train_df, viewlog_texts, how='left', on='user_id')
test_df = pd.merge(test_df, viewlog_texts, how='left', on='user_id')

In [60]:
#generate view_log count per user
viewlog_count_peruser = viewlog_merged.groupby('user_id').size().reset_index()
viewlog_count_peruser.columns =['user_id','viewlog_count_peruser']
train_df = pd.merge(train_df, viewlog_count_peruser, how='left', on='user_id')
test_df = pd.merge(test_df, viewlog_count_peruser, how='left', on='user_id')

In [61]:
print(viewlog_merged.sort_values(['user_id','session_id']).head(100))

                 server_time device_type  session_id  user_id  item_id  \
2627295  2018-12-04 07:19:00     android       86373        0   116073   
1355010  2018-11-11 10:09:00     android      143955        0    89373   
1355056  2018-11-11 10:10:00     android      143955        0    10379   
1355084  2018-11-11 10:10:00     android      143955        0    10379   
1355117  2018-11-11 10:11:00     android      143955        0    89373   
1355242  2018-11-11 10:13:00     android      143955        0    40039   
1355271  2018-11-11 10:13:00     android      143955        0    40039   
1355329  2018-11-11 10:14:00     android      143955        0    88013   
1355387  2018-11-11 10:15:00     android      143955        0     2715   
1355488  2018-11-11 10:16:00     android      143955        0     2715   
1355526  2018-11-11 10:17:00     android      143955        0   116073   
1355595  2018-11-11 10:18:00     android      143955        0    41184   
1355665  2018-11-11 10:19:00     andro

In [62]:
viewlog_sessioncount_peruser = viewlog_merged.groupby(['user_id'])['session_id']\
                                                .agg(['nunique']).reset_index()
print(viewlog_sessioncount_peruser.head(10))

   user_id  nunique
0        0       11
1        1        3
2        2       37
3        3        1
4        4        1
5        5        1
6        6        4
7        7        4
8        8       34
9        9        5


In [63]:
viewlog_merged['server_time'].head()

0    2018-10-15 08:58:00
1    2018-10-15 08:58:00
2    2018-10-15 08:58:00
3    2018-10-15 08:58:00
4    2018-10-15 08:58:00
Name: server_time, dtype: object

In [64]:
# viewlog_cols = [col for col in train_df.columns if ('viewlog_sessioncount' in col)
#                or ('viewlog_daycount_' in col) ]
# train_df.drop(viewlog_cols,inplace=True,axis=1)
# test_df.drop(viewlog_cols,inplace=True,axis=1)

In [65]:
#Number of sessions for each user in view log
viewlog_sessioncount_peruser = viewlog_merged.groupby(['user_id'])['session_id']\
                                                .agg(['nunique']).reset_index()
viewlog_sessioncount_peruser.columns =['user_id','viewlog_sessioncount_peruser']
train_df = pd.merge(train_df, viewlog_sessioncount_peruser, how='left', on='user_id')
test_df = pd.merge(test_df, viewlog_sessioncount_peruser, how='left', on='user_id')

print('viewlog_sessioncount_peruser complete')

#Number of active days for each user in view log
orig_date_format = '%Y-%m-%d %H:%M:%S'
viewlog_merged['server_time']= pd.to_datetime(viewlog_merged['server_time'],format=orig_date_format)

viewlog_merged['server_day'] = viewlog_merged['server_time'].dt.day
viewlog_daycount_peruser = viewlog_merged.groupby(['user_id'])['server_day']\
                                                .agg(['nunique']).reset_index()
viewlog_daycount_peruser.columns =['user_id','viewlog_daycount_peruser']
train_df = pd.merge(train_df, viewlog_daycount_peruser, how='left', on='user_id')
test_df = pd.merge(test_df, viewlog_daycount_peruser, how='left', on='user_id')
print('viewlog_daycount_peruser complete')

train_df['viewlog_sessioncount_peruser'] = train_df['viewlog_sessioncount_peruser'].fillna(0)
test_df['viewlog_sessioncount_peruser'] = test_df['viewlog_sessioncount_peruser'].fillna(0)
train_df['viewlog_daycount_peruser'] = train_df['viewlog_daycount_peruser'].fillna(-1)
test_df['viewlog_daycount_peruser'] = test_df['viewlog_daycount_peruser'].fillna(-1)

train_df['viewlog_sessionperday'] =train_df['viewlog_sessioncount_peruser'] / train_df['viewlog_daycount_peruser']
test_df['viewlog_sessionperday'] = test_df['viewlog_sessioncount_peruser'] / test_df['viewlog_daycount_peruser']
print('viewlog_sessionperday complete')


viewlog_sessioncount_peruser complete
viewlog_daycount_peruser complete
viewlog_sessionperday complete


In [66]:
#Number of sessions for each user in view log
viewlog_devicetype_peruser = viewlog_merged.groupby(['user_id'])['device_type']\
                                                .agg(['first']).reset_index()
viewlog_devicetype_peruser.columns =['user_id','viewlog_devicetype_peruser']
train_df = pd.merge(train_df, viewlog_devicetype_peruser, how='left', on='user_id')
test_df = pd.merge(test_df, viewlog_devicetype_peruser, how='left', on='user_id')

le =LabelEncoder()
le.fit(train_df['viewlog_devicetype_peruser'])
train_df['viewlog_devicetype_peruser'] = le.transform(train_df['viewlog_devicetype_peruser']) 
test_df['viewlog_devicetype_peruser'] = le.transform(test_df['viewlog_devicetype_peruser']) 
print(train_df['viewlog_devicetype_peruser'].unique())
print(test_df['viewlog_devicetype_peruser'].unique())


[0 1 2]
[0 1 2]


In [67]:
#convert cols to 'categorical' for model
for df in [train_df,test_df]:
    df['os_version'] = df['os_version'].astype('category')
    df['viewlog_devicetype_peruser'] = df['viewlog_devicetype_peruser'].astype('category')

In [68]:
# train_df.head()

# temp = train_df['tfidf_dense_item_id'].tolist()
# temp_sparse = csr_matrix(temp)

# print(temp_sparse[0:5])

In [69]:
# #Stats features on item price
# viewlog_price = viewlog_merged.groupby(['user_id'])['item_price'].agg(['mean','std','min','max'])
# print(viewlog_price.head())

# viewlog_price.columns = ['item_price_mean','item_price_std','item_price_min',
#                         'item_price_max']

# train_df = pd.merge(train_df, viewlog_price, how='left', on='user_id')
# test_df = pd.merge(test_df, viewlog_price, how='left', on='user_id')

In [70]:
# t0 = train_df.loc[train_df[targetcol] == 0]
# t1 = train_df.loc[train_df[targetcol] == 1]
# exclude_cols =['impression_id','os_version',
#                'impression_time','impression_date','dateblocknum',
#                'item_price_std','item_price_min', 'item_price_max',
#                targetcol]
# features = [col for col in train_df.columns if col not in exclude_cols]

# #add target enc
# features += [targetenc_col]

# print(len(features))
# print(features)

In [71]:
# t0 = train_df.loc[train_df[targetcol] == 0]
# t1 = train_df.loc[train_df[targetcol] == 1]

tfidf_name_cols = [col for col in train_df.columns if ('tfidf_' in col) & ('_name_top' in col)]
tfidf_item_price_bin_cols = [col for col in train_df.columns 
                             if ('tfidf_dense_item_price_bin' in col)]
print('len of tfidf_item_price_bin_cols:',len(tfidf_item_price_bin_cols))
print('tfidf_item_price_bin_cols:',tfidf_item_price_bin_cols)

exclude_cols =['impression_id',
#                'os_version',
               'impression_time','impression_date','dateblocknum',
               'item_price_std','item_price_min', 'item_price_max',
               targetcol,
               'impression_dayofweek',
              'viewlog_count_peruser', 'viewlog_sessioncount_peruser', 
               'viewlog_daycount_peruser', 'viewlog_sessionperday'
              ]
exclude_cols += tfidf_name_cols
exclude_cols += tfidf_item_price_bin_cols
features = [col for col in train_df.columns if col not in exclude_cols]

#add target enc
# features += [targetenc_col]

print(len(features))
print(features)


len of tfidf_item_price_bin_cols: 11
tfidf_item_price_bin_cols: ['tfidf_dense_item_price_bin3', 'tfidf_dense_item_price_bin1', 'tfidf_dense_item_price_bin7', 'tfidf_dense_item_price_bin4', 'tfidf_dense_item_price_bin5', 'tfidf_dense_item_price_bin2', 'tfidf_dense_item_price_bin8', 'tfidf_dense_item_price_bin6', 'tfidf_dense_item_price_bin0', 'tfidf_dense_item_price_bin10', 'tfidf_dense_item_price_bin9']
463
['app_code', 'impression_day', 'impression_hour', 'impression_month', 'impression_year', 'is_4G', 'os_version', 'user_id', 'tfidf_dense_category_16', 'tfidf_dense_category_114', 'tfidf_dense_category_15', 'tfidf_dense_category_111', 'tfidf_dense_category_19', 'tfidf_dense_category_18', 'tfidf_dense_category_11', 'tfidf_dense_category_113', 'tfidf_dense_category_17', 'tfidf_dense_category_13', 'tfidf_dense_category_14', 'tfidf_dense_category_115', 'tfidf_dense_category_10', 'tfidf_dense_category_116', 'tfidf_dense_category_12', 'tfidf_dense_category_117', 'tfidf_dense_category_112', 

In [72]:
# #Filter train data to exclude records which do not have lags
# train_df_copy = train_df.copy()
# firstblock = train_df['dateblocknum'].min()
# MIN_LAG = 7
# train_df = train_df[train_df['dateblocknum']>(firstblock+MIN_LAG)]

In [73]:
param = {
         'num_leaves': 127,#31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
#          'objective':'None',
         'max_depth': -1,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.3,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'AUC',
         "scale_pos_weight": 2,
#          "is_unbalance": True,
         "lambda_l1": 0.1,
#          "lambda_l1": 0.7,
#          "lambda_l2": 0.7,
         "verbosity": -1,
         "nthread": 4,
         'n_estimators' : 10000,
         "random_state": 4590,
#         "device": 'gpu',
#         'gpu_platform_id': 0,
#         'gpu_device_id': 0
}


early_stopping_rounds = 300

In [74]:
#  param=  {
#     'colsample_bytree':0.759164331,
#     'min_child_samples':85,
#     'num_leaves':20,
#     'reg_alpha':0.155214401,
#     'reg_lambda':0.594730243,
#     'subsample':0.867816493,
#     'subsample_for_bin':150000,
#     'learning_rate': 0.01,
#     'boosting': 'gbdt',
#     'bagging_seed': 2019,
#     'bagging_freq': 2,
#     'min_data_in_bin': 100,
#     'n_estimators': 10000,
#     'objective': 'binary',
#     'metric': 'None',
#     'random_state': 2333,
#     'max_depth': 15,
#     'scale_pos_weight': 2
#     }

In [75]:
early_stopping_rounds = 300

In [76]:
def lgb_logle_obj(preds, train_data):
    labels = train_data.get_label()
    residual = (labels - preds).astype("float")
    
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad_part = preds - labels
    
    grad = np.where(residual>0.7, 10.0*grad_part, grad_part)
    hess_part = preds * (1.0 - preds)
    hess = np.where(residual>0.7, 10.0*hess_part, hess_part)
    return grad, hess

def lgb_logle(preds, train_data):
    labels = train_data.get_label()
    # return a pair metric_name, result
    # since preds are margin(before logistic transformation, cutoff at 0)
    
    residual = (labels - preds).astype("float")
    
#     optcutoff,f1score = get_opt_cutoff_fbeta(labels,preds)
#     optcutoff =0.0
    loss_part = float(sum(labels != (preds > 0.0))) / len(labels)
#     loss_part = (labels != (preds > 0.0)).astype("float")

    loss = np.where(residual>0.7, 10.0*loss_part, loss_part) 

    return 'log likelihood error', np.mean(loss), False
def lgb_sk_logle(labels, preds):
    residual = (labels - preds).astype("float")
    loss_part = float(sum(labels != (preds > 0.0))) / len(labels)
    loss = np.where(residual>0.7, 10.0*loss_part, loss_part) 
  
    return 'log likelihood error', np.mean(loss), False

def get_f1loss_score(labels,preds):
    optcutoff,f1score = get_opt_cutoff_prec(labels,preds)
    f1loss = 1 - f1score
    return f1loss

def lgb_fbeta(preds, train_data):
    labels = train_data.get_label()

    optcutoff,fscore = get_opt_cutoff_fbeta(labels,preds,beta=100)
    return 'fbeta_score', fscore, True

def lgb_fbeta_auc(preds, train_data):
    labels = train_data.get_label()

    optcutoff,fscore = get_opt_cutoff_fbeta(labels,preds,beta=100)
    aucscore = roc_auc_score(labels,preds)
    return 'fbeta_auc_score', fscore+aucscore, True

def lgb_f1_auc(preds, train_data):
    labels = train_data.get_label()
    optcutoff,f1score = get_opt_cutoff_prec(labels,preds)
    aucscore = roc_auc_score(labels,preds)
    return 'f1auc_score', f1score+aucscore, True

def lgb_f1(preds, train_data):
    labels = train_data.get_label()
#     preds = preds.reshape(-1, 2)
#     print('preds before :',preds[0:10])
#     preds = preds.argmax(axis = 1)
#     preds = preds.min(axis = 1)
#     f1score = f1_score(preds, labels, average = 'micro')
    optcutoff,f1score = get_opt_cutoff_prec(labels,preds)
    
#     bscore = brier_score_loss(labels,preds) 
#     bss = 1- (bscore/0.010944928684627575) 
    return 'f1_score', f1score, True

def convert_probtolabels(preds,cutoff=0.5):
    y_bin= preds.copy()
    y_bin[preds>cutoff] = 1
    y_bin[preds<=cutoff] = 0
    y_bin=y_bin.astype(int)

    return y_bin

def get_opt_cutoff_fbeta(labels,preds,beta=5):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    beta_square = beta**2
    f_score= (1+beta_square)*((precision*recall)/(beta_square*precision+recall))
    optimal_idx = np.nanargmax(f_score)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold, f_score[optimal_idx]

def get_opt_cutoff_prec(labels,preds):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    f1_score= 2*((precision*recall)/(precision+recall))
    optimal_idx = np.nanargmax(f1_score)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold, f1_score[optimal_idx]
def computef1scoreandconfmatrix(y_true,y_preds):    
    opt_cutoff, f1score = get_opt_cutoff_prec(y_true,y_preds)
    print('opt_cutoff:',opt_cutoff)
    print('f1 score:',f1score)
    pred_labels = convert_probtolabels(y_preds,cutoff=opt_cutoff)
    print('conf matrix:',confusion_matrix(y_true,pred_labels))
    return opt_cutoff,f1score,pred_labels

In [77]:
# train_df_copy = train_df.copy()
# test_df_copy = test_df.copy()
# target_copy = target.copy()

In [78]:
# #temporary test split 
# random_fold = 0

# folds = StratifiedKFold(n_splits=5, shuffle=False, random_state=1000)
# for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    
#     if fold_!=random_fold:
#         continue
#     new_train = train_df.iloc[trn_idx]
#     new_test =  train_df.iloc[val_idx]
#     new_target = target.iloc[trn_idx]
#     test_target = target.iloc[val_idx]
#     break
    

# train_df = new_train.copy()
# test_df = new_test.copy()
# target=new_target.copy()

# print(train_df.shape,test_df.shape)

In [79]:
# train_df = train_df_copy.copy()
# test_df = test_df_copy.copy()
# target  = target_copy.copy()

In [80]:
def runlgb(train_df,test_df,target,n_splits=5, random_state=888):
    
    folds = StratifiedKFold(n_splits=n_splits, shuffle=False, random_state=random_state)
    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))
    feature_importance_df = pd.DataFrame()
    valid_scores =[]
    iterations=[]
    
    test_cur = test_df.copy()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):

        train_cur = train_df.iloc[trn_idx]
        val_cur = train_df.iloc[val_idx]
        target_cur = target.iloc[trn_idx]
        target_val_cur = target.iloc[val_idx]

        print("Fold {}".format(fold_))
        trn_data = lgb.Dataset(train_cur[features], label=target_cur)
        val_data = lgb.Dataset(val_cur[features], label=target_val_cur)

        num_round = 1000000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], 
                        verbose_eval=100, 
#                         feval=lgb_logle,
                        early_stopping_rounds = early_stopping_rounds)
        oof[val_idx] = clf.predict(val_cur[features], num_iteration=clf.best_iteration)
        computef1scoreandconfmatrix(target_val_cur,oof[val_idx]) 
        valscore = roc_auc_score(target_val_cur,oof[val_idx])
        print("AUC score: {:<8.5f}".format(valscore))
        valid_scores += [valscore]
        iterations += [clf.best_iteration]

#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["Feature"] = features
#         fold_importance_df["importance"] = clf.feature_importance()
#         fold_importance_df["fold"] = fold_ + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(test_cur[features], num_iteration=clf.best_iteration) / folds.n_splits

    print()
    print("CV AUC score: {:<8.5f}".format(roc_auc_score(target, oof)))
    print('valid scores: ',valid_scores)
    opt_cutoff, f1score, oof_labels = computef1scoreandconfmatrix(target,oof)
    predictions_labels = convert_probtolabels(predictions,cutoff=opt_cutoff)
    
    del test_cur
    
    return oof,predictions,oof_labels, predictions_labels,feature_importance_df,iterations

In [81]:
def runlgb_targetenc(train_df,test_df,target,n_splits=5, random_state=888):
    
    folds = StratifiedKFold(n_splits=n_splits, shuffle=False, random_state=random_state)
    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))
    feature_importance_df = pd.DataFrame()
    valid_scores =[]
    test_cur = test_df.copy()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):

        train_cur = train_df.iloc[trn_idx]
        val_cur = train_df.iloc[val_idx]
        target_cur = target.iloc[trn_idx]
        target_val_cur = target.iloc[val_idx]
        
        #Concat current encoding train, valid and test files
#         print('val shape bef:',val_cur.shape)
#         print('tr shape bef:',train_cur.shape)
#         print('test shape bef:',test_cur.shape)
        
        
        train_cur=pd.concat([train_cur,tr_encs[fold_]],axis=1)
        val_cur=pd.concat([val_cur,val_encs[fold_]],axis=1)
        test_cur=pd.concat([test_df,test_encs[fold_]],axis=1)        
        
#         print('val shape after:',val_cur.shape)
#         print('tr shape after:',train_cur.shape)
#         print('test shape after:',test_cur.shape)
       

        print("Fold {}".format(fold_))
        trn_data = lgb.Dataset(train_cur[features], label=target_cur)
        val_data = lgb.Dataset(val_cur[features], label=target_val_cur)

        num_round = 1000000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], 
                        verbose_eval=100, 
#                         feval=lgb_logle,
                        early_stopping_rounds = early_stopping_rounds)
        oof[val_idx] = clf.predict(val_cur[features], num_iteration=clf.best_iteration)
        computef1scoreandconfmatrix(target_val_cur,oof[val_idx]) 
        valscore = roc_auc_score(target_val_cur,oof[val_idx])
        print("AUC score: {:<8.5f}".format(valscore))
        valid_scores += [valscore]

#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["Feature"] = features
#         fold_importance_df["importance"] = clf.feature_importance()
#         fold_importance_df["fold"] = fold_ + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(test_cur[features], num_iteration=clf.best_iteration) / folds.n_splits

    print()
    print("CV AUC score: {:<8.5f}".format(roc_auc_score(target, oof)))
    print('valid scores: ',valid_scores)
    opt_cutoff, f1score, oof_labels = computef1scoreandconfmatrix(target,oof)
    predictions_labels = convert_probtolabels(predictions,cutoff=opt_cutoff)
    
    del test_cur
    
    return oof,predictions,oof_labels, predictions_labels,feature_importance_df

In [82]:
from catboost import CatBoostClassifier

In [83]:
def runcatboost(train_df,test_df,target,n_splits=5, random_state=888):
    
    folds = StratifiedKFold(n_splits=n_splits, shuffle=False, random_state=random_state)
    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))
    feature_importance_df = pd.DataFrame()
    valid_scores =[]
    
    test_cur = test_df.copy()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):

        train_cur = train_df.iloc[trn_idx]
        val_cur = train_df.iloc[val_idx]
        target_cur = target.iloc[trn_idx]
        target_val_cur = target.iloc[val_idx]

        print("Fold {}".format(fold_))
        clf = CatBoostClassifier(
            iterations=param['n_estimators'],
            learning_rate=0.01,
#             eval_metric=['auc'],
            early_stopping_rounds = 200,
            task_type='GPU'
          )
  
        clf.fit(
                train_cur[features], target_cur,
                eval_set=(val_cur[features], target_val_cur),
                use_best_model=True,
                cat_features =[0],
                verbose=100
          )          
        

        num_round = 1000000
        oof[val_idx] = clf.predict_proba(val_cur[features])[:,1]
        computef1scoreandconfmatrix(target_val_cur,oof[val_idx]) 
        valscore = roc_auc_score(target_val_cur,oof[val_idx])
        print("AUC score: {:<8.5f}".format(valscore))
        valid_scores += [valscore]

#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["Feature"] = features
#         fold_importance_df["importance"] = clf.feature_importance()
#         fold_importance_df["fold"] = fold_ + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict_proba(test_cur[features])[:,1] / folds.n_splits

    print()
    print("CV AUC score: {:<8.5f}".format(roc_auc_score(target, oof)))
    print('valid scores: ',valid_scores)
    opt_cutoff, f1score, oof_labels = computef1scoreandconfmatrix(target,oof)
    predictions_labels = convert_probtolabels(predictions,cutoff=opt_cutoff)
    
    del test_cur
    
    return oof,predictions,oof_labels, predictions_labels,feature_importance_df

In [84]:
%%time
param['n_estimators'] = 10000
oof,predictions,oof_labels, predictions_labels,feature_importance_df,iterations = \
                runlgb(train_df,test_df,target)

Fold 0
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.833202	valid_1's auc: 0.722522
[200]	training's auc: 0.863014	valid_1's auc: 0.726936
[300]	training's auc: 0.881305	valid_1's auc: 0.73137
[400]	training's auc: 0.897526	valid_1's auc: 0.732732
[500]	training's auc: 0.910459	valid_1's auc: 0.733415
[600]	training's auc: 0.921552	valid_1's auc: 0.733851
[700]	training's auc: 0.930603	valid_1's auc: 0.73319
[800]	training's auc: 0.937378	valid_1's auc: 0.732442
Early stopping, best iteration is:
[599]	training's auc: 0.921472	valid_1's auc: 0.733865
opt_cutoff: 0.14090556564020437
f1 score: 0.18972332015810275
conf matrix: [[40653  4697]
 [ 1454   719]]
AUC score: 0.73387 
Fold 1
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.831954	valid_1's auc: 0.713971
[200]	training's auc: 0.8623	valid_1's auc: 0.725093
[300]	training's auc: 0.882677	valid_1's auc: 0.728756
[400]	training's auc: 0.897889	valid_1's au

In [85]:
def runlgbfull(train_df,test_df,target,n_splits=5, random_state=888):
    
    predictions = np.zeros(len(test_df))
    
    test_cur = test_df.copy()

    trn_data = lgb.Dataset(train_df[features], label=target)

    num_round = param['n_estimators']
    clf = lgb.train(param, trn_data, num_round)

    predictions = clf.predict(test_cur[features]) 
    
    del test_cur
    
    return oof,predictions,oof_labels, predictions_labels,feature_importance_df

In [86]:
# oof1 = oof.copy()

In [87]:
np.save('oof.npy',oof)
np.save('predictions.npy',predictions)

In [88]:
sub_df = pd.DataFrame({"impression_id":test_df["impression_id"].values})
sub_df[targetcol] = predictions
print(sub_df.head())
sub_df.to_csv("submission.csv", index=False)

                      impression_id  is_click
0  a9e7126a585a69a32bc7414e9d0c0ada  0.039047
1  caac14a5bf2ba283db7708bb34855760  0.051323
2  13f10ba306a19ce7bec2f3cae507b698  0.113446
3  39c4b4dc0e9701b55a0a4f072008fb3f  0.040985
4  bf5a572cca75f5fc67f4b14e58b11d70  0.259132


In [89]:
%%time
iterations_mean = np.mean(np.array(iterations))
param['n_estimators'] = int(iterations_mean)
oof,predictions,oof_labels, predictions_labels,feature_importance_df = \
                runlgbfull(train_df,test_df,target)

CPU times: user 6min 6s, sys: 23.7 s, total: 6min 30s
Wall time: 3min 38s


In [90]:
sub_df = pd.DataFrame({"impression_id":test_df["impression_id"].values})
sub_df[targetcol] = predictions
print(sub_df.head())
sub_df.to_csv("submission_full.csv", index=False)

                      impression_id  is_click
0  a9e7126a585a69a32bc7414e9d0c0ada  0.039646
1  caac14a5bf2ba283db7708bb34855760  0.037509
2  13f10ba306a19ce7bec2f3cae507b698  0.134156
3  39c4b4dc0e9701b55a0a4f072008fb3f  0.040773
4  bf5a572cca75f5fc67f4b14e58b11d70  0.249519


In [91]:
# cols = (feature_importance_df[["Feature", "importance"]]
#         .groupby("Feature")
#         .mean()
#         .sort_values(by="importance", ascending=False)[:150].index)
# best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

# plt.figure(figsize=(14,28))
# sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
# plt.title('Features importance (averaged/folds)')
# plt.tight_layout()
# plt.savefig('FI.png')